### Basic ChatCompletion API
The purpsoe of the completion API is to provide a response to complete a prompt. For more information, see docs: https://platform.openai.com/docs/api-reference/chat/create

Let's call the ChatCompletion API using gpt-3.5-turbo. 

In [ ]:
pip install openai && pip install python-dotenv

In [3]:
from openai import OpenAI
from dotenv import dotenv_values

In [4]:
# Set the Open API key from the .env file
config = dotenv_values(".env")
client = OpenAI(
  api_key=config["OPEN_API_KEY"]
)

In [35]:
# Create a completion model using gpt-3.5-turbo
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    { "role": "user", "content": "The dog says " } # role = "user" means you or who is chatting or who is asking the question
  ],
)

In [39]:
print(completion)

ChatCompletion(id='chatcmpl-9vBVSY2lkCyj5gxp7fyc3P71pmn4B', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='"woof"', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1723415954, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=4, prompt_tokens=11, total_tokens=15))


### Analysis of ChatCompletion API
The result from the prompt "The dog says " was "woof" from the gpt-3.5-turbo (defaulted to gpt-3.5-turbo-0125) model. If you check your OpenAI API usage, you'll see the billing usage of the API.

As of today (08/11/2024), the pricing for gpt-3.5-turbo-0125 is `$0.50 / 1M tokens` for input and `$1.50 / 1M tokens` for output.

See billing usage in the website https://platform.openai.com/organization/usage.

### max_tokens parameter
The max_tokens parameter specifies the maximum number of tokens in the response. For more information about tokens, read the Tokens.md file.

In [7]:
# Text completion uses max_tokens of 16 in the response
# See docs: https://platform.openai.com/docs/api-reference/completions/create#completions-create-max_tokens
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    { "role": "user", "content": "The top 10 places to visit are: " }
  ],
  max_tokens=16
)

In [8]:
print(completion)

ChatCompletion(id='chatcmpl-9vBxl7G2L4swedDyFOmMQcFa3soj5', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='1. The Grand Canyon, Arizona, USA\n2. Machu Picchu', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1723417709, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=16, prompt_tokens=17, total_tokens=33))


### Analysis of max_tokens
We asked the model to predict the top 10 places to visit, but it only gave us 2 places. This is because we set the `max_tokens=16` in the completions API request, so the model stopped prematurely with a `finish_reason` of 'length'.